## Mahindra First Choice Data Analysis

### Problem Statements
    1. Geolocation Based Customer Analysis
        * Explore how various factors like make, model, time & type of service, etc vary with location.
            * Identifying the ownership pattern of cars throughout the country.
            * This also captures the problem wherein information regarding the spending patterns can be identified.
        * Identify the type of order each state receives and present it as an interactive visualization.
    2. Market Segmentation
        * Divide market of potential customers into internally homogeneous and mutually heterogeneous groups or segments.
            * To be based on different characteristics captured in the data.
     3. Customer Lifetime value prediction

#### Further Problem Statements
    1. Inventory Management and Recommendation
    2. Marketing Recommendation
    3. Customer Prediction

### The Data

* Dataset contains the following:
    1. Customer Data
    2. Invoice Data
    3. Joint Technical Data
    4. Plant Master

### Data Cleaning and Feature Engineering

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import string
import sys
import time
import warnings

from matplotlib import pyplot as plt
from scipy import stats
from my_utils import *

warnings.filterwarnings("ignore")
np.set_printoptions(threshold=sys.maxsize)

In [ ]:
customer_data = pd.read_excel("redacted/data/Customer_Data.xlsx", sheet_name='Sheet1')

In [ ]:
invoice = pd.read_csv("redacted/data/Final_invoice.csv")

In [ ]:
jtd = pd.read_csv("redacted/data/JTD.csv")

In [ ]:
plant_master = pd.read_excel("redacted/data/Plant Master.xlsx", sheet_name='Sheet1')

### Customer Data

In [ ]:
check_null(customer_data)

* <code>['Death date', 'Date of Birth', 'Occupation', 'Marital Status']</code> have a high number of missing values which would not be possible to impute. Hence dropping these columns.
* <code>['Partner Type', 'Data Origin']</code> have too few a number of missing values which may not justify the time and effort to be put in for diminished gains. Hence dropping the entries where these columns have missing values.

In [ ]:
if 'Occupation' in customer_data.columns:
    customer_data.drop(columns=['Death date', 'Date of Birth', 'Occupation', 'Marital Status'], axis=1, inplace=True)
if 'Partner Type' in customer_data.columns:
    customer_data.drop(customer_data[customer_data['Partner Type'].isnull() | customer_data['Data Origin'].isnull()].index,
                       axis=0, inplace=True)

In [ ]:
check_null(customer_data)

In [ ]:
_ = customer_data['Title'].value_counts()
sns.barplot(x=_.values, y=_.index)

* The <code>Title</code> only contains <code>['Mr.', 'Mrs.']</code>. <code>['Company', 'Mr. and Mrs.']</code> weren't found.
* They have an imbalanced distribution.
* There seems to be no features that would aid in predicting the <code>Title</code> before merging.

In [ ]:
titles = ['Mr.', 'Mrs.']
customer_data['Title'] = np.where(customer_data['Title'].isnull(),
                         titles[math.floor(random.random()*((483931 + 25094)/483931))], 
                         customer_data['Title'])

* All values in `customer_data` have been imputed.

In [ ]:
customer_data['Business Partner'].nunique()
len(customer_data)

In [12]:
customer_data['Customer No.'].nunique()
len(customer_data)

555202

555202

* <code>['Customer No.', 'Business Partner']</code> seem to be some sort of identifiers for internal and external systems.
* Both contain all unique values with no repetition.
* They do not offer any prediction power, but may aid in merging.

In [13]:
customer_data = pd.get_dummies(customer_data, columns=['Partner Type', 'Data Origin', 'Title'])

In [14]:
customer_data.head()

,Business Partner,Customer No.,Partner Type_1.0,Partner Type_2.0,Partner Type_3.0,Partner Type_4.0,Partner Type_9001.0,Partner Type_9002.0,Partner Type_9003.0,Data Origin_Z001,...,Data Origin_Z016,Data Origin_Z017,Data Origin_Z018,Data Origin_Z019,Data Origin_Z020,Data Origin_Z021,Data Origin_Z022,Title_1.0,Title_2.0,Title_Mr.
0,31,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
1,32,2,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,34,4,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,35,5,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,42,7,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
